### 百度图片爬取(一次最多只有五十几张图片)

In [10]:
import re
import requests

folder= 'images'
if os.path.exists(folder)==False:
    os.mkdir(folder)

def dowmloadPic(html, keyword):
    pic_url = re.findall('"objURL":"(.*?)",', html, re.S)
    i = 1
    print('找到关键词:' + keyword + '的图片，现在开始下载图片...')
    for each in pic_url:
        print('正在下载第' + str(i) + '张图片，图片地址:' + str(each))
        try:
            pic = requests.get(each, timeout=10)
        except requests.exceptions.ConnectionError:
            print('【错误】当前图片无法下载')
            continue

        dir = folder + keyword + '_' + str(i) + '.jpg'
        fp = open(dir, 'wb')
        fp.write(pic.content)
        fp.close()
        i += 1


if __name__ == '__main__':
    word = input("Input key word: ")
    url = 'http://image.baidu.com/search/flip?tn=baiduimage&ie=utf-8&word=' + word + '&ct=201326592&v=flip'
    result = requests.get(url)
    dowmloadPic(result.text, word)

Input key word: 123
找到关键词:123的图片，现在开始下载图片...
正在下载第1张图片，图片地址:http://downza.img.zz314.com/soft/grab/20170801/88489370-622b-49ca-a492-9051992698a3contentImg1.png
正在下载第2张图片，图片地址:http://img.25pp.com/uploadfile/youxi/images/2015/0720/20150720082110467.jpg
【错误】当前图片无法下载
正在下载第2张图片，图片地址:http://www.suntop168.com/userfiles/vi-201402/DEF8AE6C.gif
正在下载第3张图片，图片地址:http://sanmugroup.com.cn/img.php?www.jhvis.cn/ueditor/../uploadfiles/images/20131207180150485048.jpg


KeyboardInterrupt: 

### 爬取彼岸桌面网站图片(一直爬网站所有相关图片)

In [13]:
import requests
from bs4 import BeautifulSoup
import os

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36'}
for i in range(2, 200):
    list_url = 'http://www.netbian.com/meinv/index_'+str(i)+'.htm'
    list_source = requests.get(list_url, headers=headers).text
    list_soup = BeautifulSoup(list_source, 'html.parser')
    page_urls = list_soup.find('div', {'class': 'list'}).find_all('li')
    for j in page_urls:
        second_url = 'http://www.netbian.com'+j.find('a')['href']
        try:
            second_source = requests.get(second_url, headers=headers).text
        except:
            print(' ')
        second_soup = BeautifulSoup(second_source, 'html.parser')
        second_page_urls = second_soup.find('div', {'class': 'pic'}).find_all('p')
        for each in second_page_urls:
            third_url = 'http://www.netbian.com'+each.find('a')['href']
            # print(third_url)
            third_request = requests.get(third_url,headers = headers)
            third_pagesource = third_request.text
            third_soup = BeautifulSoup(third_pagesource,'html.parser')
            img_name = third_soup.find('title').text
            # print(img_name)
            try:
              img_source = third_soup.find('table', {'id': 'endimg'}).find_all('img')#找到图片标签
            except:
                print(' ')
            # print(img_source)
            for img in img_source:
                url = img['src']
                print(url)
                img = requests.get(url, headers=headers).content
                filename = folder+'\\'+str(k) + '.jpg'
                f = open(filename, 'wb')
                f.write(img)
                f.close()
                k+=1

http://img.netbian.com/file/2019/1009/b2e2f2d3b497922e7249364d972a8de3.jpg
http://img.netbian.com/file/2019/1009/ccc79ab72ab94cf067917a4a83766ae2.jpg
 
http://img.netbian.com/file/2019/1009/ccc79ab72ab94cf067917a4a83766ae2.jpg
http://img.netbian.com/file/2019/0910/7efdc8e60d36329c404005c3c34f7af8.jpg
http://img.netbian.com/file/2019/0905/a5c693ca2a2d8746583d16353f45c0b3.jpg
http://img.netbian.com/file/2019/0905/b345d6c2c1741e584cb98d200e451a56.jpg
http://img.netbian.com/file/2019/0905/8520ae74b84b193f00fd16890778a4bc.jpg
http://img.netbian.com/file/2019/0905/57f0619990162e464d73bc04c1179de6.jpg
http://img.netbian.com/file/2019/0722/46a77e637238b439e445a8e11279eb28.jpg
http://img.netbian.com/file/2019/0722/3e42bd500982d800daffaad8ba0dacac.jpg
http://img.netbian.com/file/2019/0713/9abae6463e9ca21bc45bdf6d2a413c2f.jpg
http://img.netbian.com/file/2019/0708/d1cb4cb5bb710a0c0c30f531d7532524.jpg
http://img.netbian.com/file/2019/0615/9d07e39672c7f457bba4655f4b5c8b39.jpg
http://img.netbian.com/

KeyboardInterrupt: 

### 截取图片中出现的人像数据并调整尺寸到96\*96大小

In [14]:
import cv2
import sys
import os.path
from glob import glob

if os.path.exists('faces')==False:
    os.mkdir('faces')
#文件路径及文件明不能有中文以及特殊字符
def detect(filename, cascade_file="E:/Pictures/lbpcascade_animeface.xml"):
    if not os.path.isfile(cascade_file):
        raise RuntimeError("%s: not found" % cascade_file)

    cascade = cv2.CascadeClassifier(cascade_file)
    image = cv2.imread(filename)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.equalizeHist(gray)

    faces = cascade.detectMultiScale(gray,
                                     # detector options
                                     scaleFactor=1.1,
                                     minNeighbors=5,
                                     minSize=(48, 48))
    for i, (x, y, w, h) in enumerate(faces):
        face = image[y: y + h, x:x + w, :]
        face = cv2.resize(face, (96, 96))
        save_filename = '%s-%d.jpg' % (os.path.basename(filename).split('.')[0], i)
        cv2.imwrite("faces/" + save_filename, face)


if __name__ == '__main__':
    file_list = glob('images/*.jpg')
    print(len(file_list))
    for filename in file_list:
        print(filename)
        detect(filename)


34
images\3537.jpg
images\3538.jpg
images\3539.jpg
images\3540.jpg
images\3541.jpg
images\3542.jpg
images\3543.jpg
images\3544.jpg
images\3545.jpg
images\3546.jpg
images\3547.jpg
images\3548.jpg
images\3549.jpg
images\3550.jpg
images\3551.jpg
images\3552.jpg
images\3553.jpg
images\3554.jpg
images\3555.jpg
images\3556.jpg
images\3557.jpg
images\3558.jpg
images\3559.jpg
images\3560.jpg
images\3561.jpg
images\3562.jpg
images\3563.jpg
images\3564.jpg
images\3565.jpg
images\3566.jpg
images\3567.jpg
images\3568.jpg
images\3569.jpg
images\3570.jpg
